In [22]:
# load environment variables
from dotenv import load_dotenv
import os
load_dotenv()

# selenium to parse dynamic web pages
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# import utilities
import pandas as pd
import time

In [23]:
# get environment variables
card_num = os.environ.get("mississauga-library-card")

# search terms
query = "banquet"
data_values = [
    581212, # Banquet Facilities
    791102, # Ballrooms
    799904, # Auditoriums
    738931] # Convention & Meeting Facilities
province_list = [
    'ON'] # Ontario

In [24]:
# define options for browser
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')

In [25]:
# load browser driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# define context for waiting on webpage to load
# if page loads, the event listener will return true
# if page loaded before executing this function, then return true
def document_initialised(driver):
    time.sleep(3)
    return driver.execute_script("if (document.readyState === 'complete') {return true;} else {window.addEventListener('load', () => {return true;});}")

# open URL in browser
driver.get("https://www.mississauga.ca/library/research-and-learn/business/")

# if the element text(ID) is equal to 'options', do nothing
# otherwise, throw an assertion error]
WebDriverWait(driver, timeout=10).until(document_initialised)
driver.find_element(By.XPATH, ("//a[@title = 'Canadian Businesses']")).click()

# check off on the ToC and Agree
WebDriverWait(driver, timeout=10).until(document_initialised)
driver.find_element(By.ID, "chkAgree").click()
driver.find_element(By.CLASS_NAME, "action-agree").click()

# enter card number and log on
WebDriverWait(driver, timeout=10).until(document_initialised)
driver.find_element(By.ID, "matchcode").send_keys(card_num)
driver.find_element(By.CLASS_NAME, "action-submit-form").click()

# open Canadian Business search
WebDriverWait(driver, timeout=30).until(document_initialised)
driver.find_element(By.CLASS_NAME, ("action-do-search")).click()

# switch to Advanced Search and interact with filters
WebDriverWait(driver, timeout=10).until(document_initialised)
driver.find_element(By.CLASS_NAME, "advancedSearch").click()
driver.find_element(By.ID, "cs-YellowPageHeadingOrSic").click() # Business Type: Keyword/SIC/NAICS
driver.find_element(By.ID, "cs-Province").click() # Geography: Province

# wait for advanced filter options to populate
WebDriverWait(driver, 10).until(document_initialised)
driver.find_element(By.CSS_SELECTOR, "input#VerifiedOnly").click()
driver.find_element(By.CSS_SELECTOR, "input#sicPrimaryOptionId").click() # radio: Search primary SIC Only
driver.find_element(By.ID, "sicLookupKeyword").send_keys(query) # search field: Query
driver.find_element(By.ID, "searchSic").click() # search button: Query

# wait for results to populate and highlight chosen query fields
WebDriverWait(driver, timeout=10).until(document_initialised)
for value in data_values:
    driver.find_element(By.XPATH, f"//li[@data-value = {value}]").click() # results: Highlight NAICS results
driver.find_element(By.XPATH, f"//li[@data-value = 'ON']").click() # results: Highlight Ontario results
driver.find_element(By.CLASS_NAME, "action-view-results").click() # View Results with filters

# results page
WebDriverWait(driver, timeout=10).until(document_initialised)
driver.find_element(By.XPATH, "//*[@alt = 'Optional Column']").click()
driver.find_element(By.XPATH, "//*[@data-key = 'Title']").click()

In [28]:
# function to scrape through each page
def scrape_data(driver):

    # find the resulting number of pages
    max_pages = driver.find_element(By.CLASS_NAME, "data-page-max").text

    # define containers
    data = list()
    heading_names = list()

    # scrape table headings
    headings = driver.find_elements(By.XPATH, "//thead[@id = 'searchResultsHeader']/tr/th")
    for heading in headings:
        heading_names.append(heading.text)

    for _ in range(1, int(max_pages)+1):
        # scrape entries from table
        rows = driver.find_elements(By.XPATH, "//tbody[@id = 'searchResultsPage']/tr")

        for row in rows:
            row_data = list()
            for col in row.find_elements(By.TAG_NAME, "td"):
                row_data.append(col.text)
            data.append(row_data)

        # go to the next page
        driver.find_elements(By.CLASS_NAME, "next")[0].click()

        # wait for next page to load
        WebDriverWait(driver, timeout=10).until(document_initialised)

    return data, heading_names

In [29]:
data, headings = scrape_data(driver)
headings

['',
 'Company Name',
 'Executive Name',
 'Street Address',
 'City, Province',
 'Postal Code',
 'Phone',
 'Title']

In [31]:
df = pd.DataFrame(data, columns=headings)

In [32]:
df.to_excel('lead-list-scraped.xlsx')